In [ ]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
from typing import Optional, Union, Tuple


import torch.nn.functional as F
from torch.distributions import Distribution
from torch.distributions import Bernoulli, Normal, StudentT, Poisson, NegativeBinomial

from torch.distributions import constraints


from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.utils import AirPassengers, AirPassengersPanel, AirPassengersStatic
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
from neuralforecast.losses.pytorch import MQLoss, DistributionLoss, MSE, MAE

In [ ]:
from numpy.random import seed
from random import randrange

In [ ]:
df = pd.read_excel("SP100_vol (clean).xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [ ]:
n_inputs = [3,5,10,15,21,42,84]
hidden_size = [60,90,120,150,180]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]
n_head=[6,9,12,15,18]
decoder_input_size_multiplier=[0.3,0.4,0.5,0.6,0.7]
dropout=[0,0.2,0.3,0.4]
patch_len=[4,6,8,10,12]
stride=[6,7,8,9,10]
encoder_layers = [1,2,3,4,5]

# Hyperparameter Search

In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 63510272


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 450 | Inputs: 3 | hidden_size: 0 | Loss: 1 | Scaler Type: minmax | attn_dropout: 0.2 | encoder_layers: 1
Learning Rate: 3 | head_dropout: 0.2 | Dropout: 0.3 | n_head: 6 | patch_len: 0
RMSE: 0.0025997283822753498 | QLIKE: 0.023873242116607483 | MAE: 0.0014405921637271606 | MAPE: 0.1425073072475016


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 45278987


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 15 | hidden_size: 4 | Loss: 2 | Scaler Type: minmax | attn_dropout: 0.4 | encoder_layers: 2
Learning Rate: 0 | head_dropout: 0.4 | Dropout: 0.3 | n_head: 18 | patch_len: 2
RMSE: 0.00254056277794981 | QLIKE: 0.023174892497986682 | MAE: 0.0014739689872967832 | MAPE: 0.14376088593078926


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 126692174


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 150 | Inputs: 3 | hidden_size: 4 | Loss: 1 | Scaler Type: standard | attn_dropout: 0 | encoder_layers: 3
Learning Rate: 2 | head_dropout: 0 | Dropout: 0.4 | n_head: 18 | patch_len: 1
RMSE: 0.0025914263778733463 | QLIKE: 0.023709140242357457 | MAE: 0.0015306022505306826 | MAPE: 0.14912474468885392


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 101682327


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 21 | hidden_size: 0 | Loss: 1 | Scaler Type: minmax | attn_dropout: 0.2 | encoder_layers: 2
Learning Rate: 3 | head_dropout: 0.2 | Dropout: 0.3 | n_head: 6 | patch_len: 1
RMSE: 0.0025576795524376995 | QLIKE: 0.023333768841070433 | MAE: 0.0015269686490010472 | MAPE: 0.1488013006216861


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 75887829


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 15 | hidden_size: 3 | Loss: 0 | Scaler Type: robust | attn_dropout: 0.2 | encoder_layers: 2
Learning Rate: 2 | head_dropout: 0.2 | Dropout: 0.2 | n_head: 15 | patch_len: 1
RMSE: 0.0025892102202141162 | QLIKE: 0.023965892596191404 | MAE: 0.0015813346821842412 | MAPE: 0.15858708841783992


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 124151285


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 15 | hidden_size: 3 | Loss: 2 | Scaler Type: standard | attn_dropout: 0.3 | encoder_layers: 3
Learning Rate: 0 | head_dropout: 0.3 | Dropout: 0 | n_head: 15 | patch_len: 4
RMSE: 0.002529506699827837 | QLIKE: 0.022942362295855584 | MAE: 0.0014807732587621629 | MAPE: 0.144198631314966


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 116946082


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 650 | Inputs: 10 | hidden_size: 3 | Loss: 3 | Scaler Type: minmax | attn_dropout: 0 | encoder_layers: 3
Learning Rate: 1 | head_dropout: 0 | Dropout: 0.3 | n_head: 15 | patch_len: 2
RMSE: 0.0026049252896022607 | QLIKE: 0.02391052561719365 | MAE: 0.001569140956236811 | MAPE: 0.15143743483217884


In [ ]:
for z in range (5):
  i=randrange(7)
  h=randrange(5)
  m=randrange(4)
  m1=randrange(4)
  a=randrange(5)
  k=randrange(9)
  l=randrange(4)
  n=randrange(4)
  o=randrange(3)
  a1=randrange(5)
  a2=randrange(5)
  a3=randrange(5)
  a4=randrange(4)
  a5=randrange(5)
  a6=randrange(4)
  model = PatchTST(h=1, input_size=n_inputs[i],
             hidden_size=hidden_size[h], linear_hidden_size =int(hidden_size[h]*2),
           dropout=dropout[m],  fc_dropout=dropout[m], head_dropout=dropout[a6],
                   patch_len=patch_len[a2],stride=stride[a2],
           attn_dropout=dropout[a6], n_heads=n_head[h],
             encoder_layers=encoder_layers[a5],loss=losses[m1],
             max_steps=epochs[k],
           learning_rate=learning_rate[n], num_lr_decays=num_lr_decays[n],
             val_check_steps=500, batch_size=32,
             scaler_type=scaler_type[o], random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | hidden_size: {h} | Loss: {m1} | Scaler Type: {scaler_type[o]} | attn_dropout: {dropout[a6]} | encoder_layers: {encoder_layers[a5]}")
  print(f"Learning Rate: {n} | head_dropout: {dropout[a6]} | Dropout: {dropout[m]} | n_head: {n_head[h]} | patch_len: {a2}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 75931618


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 50 | Inputs: 15 | hidden_size: 4 | Loss: 2 | Scaler Type: standard | attn_dropout: 0.3 | encoder_layers: 1
Learning Rate: 3 | head_dropout: 0.3 | Dropout: 0.2 | n_head: 18 | patch_len: 1
RMSE: 0.002784170765633352 | QLIKE: 0.02886488360654688 | MAE: 0.0017373163832565936 | MAPE: 0.17373532538712377


INFO:lightning_fabric.utilities.seed:Seed set to 42914685

Epoch: 150 | Inputs: 42 | hidden_size: 3 | Loss: 0 | Scaler Type: standard | attn_dropout: 0.3 | encoder_layers: 2
Learning Rate: 1 | head_dropout: 0.3 | Dropout: 0 | n_head: 15 | patch_len: 1
RMSE: 0.002526472261895962 | QLIKE: 0.02257904732598412 | MAE: 0.0015144523594421628 | MAPE: 0.15277428882298233

In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 2825471


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 48545993
RMSE: 0.0025420640429757863 | QLIKE: 0.022731880607510555 | MAE: 0.0015099478295317803 | MAPE: 0.15217609757008171


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 119221403


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 73594081
RMSE: 0.002526552682903103 | QLIKE: 0.022625196468704412 | MAE: 0.0015188277133975328 | MAPE: 0.15236499767648068


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 68632218


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 39512569
RMSE: 0.0025272656801256415 | QLIKE: 0.02257539502081998 | MAE: 0.00151464547241268 | MAPE: 0.15199847083726345


In [ ]:
for z in range (5):
  randomseed=randrange(129228148)
  model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
                random_seed=randomseed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "PatchTST-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["PatchTST"].values
  else:
    Y_hat=forecasts["PatchTST-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Random Seed: {randomseed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 38034720


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Random Seed: 96692183
RMSE: 0.002534576425855726 | QLIKE: 0.022562283553832167 | MAE: 0.0015023772234653348 | MAPE: 0.15010754656635802


# Test

In [ ]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# H=1

In [ ]:
model = PatchTST(h=1, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=1,static_df=df_static,n_windows=None, test_size=test_length-test_length%1,step_size=1)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.0038693403515680717 | QLIKE: 0.03096300015291199 | MAE: 0.002454087556661601 | MAPE: 0.19106431440025265


In [ ]:
if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

In [ ]:
df2.to_excel("PatchTST forecasts.xlsx")

# H=5

In [ ]:
H=5
model = PatchTST(h=H, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004646340034091326 | QLIKE: 0.04348540255877384 | MAE: 0.0029313563208999472 | MAPE: 0.22859590782445732


# H=10

In [ ]:
H=10
model = PatchTST(h=H, input_size=42,
             hidden_size=hidden_size[3], linear_hidden_size =int(hidden_size[3]*2),
           dropout=dropout[0],  fc_dropout=dropout[0], head_dropout=0.3,
                   patch_len=patch_len[1],stride=stride[1],
           attn_dropout=0.3, n_heads=15,
             encoder_layers=2,loss=losses[0],
             max_steps=150,
           learning_rate=learning_rate[1], num_lr_decays=num_lr_decays[1],
             val_check_steps=500, batch_size=32,
             scaler_type='standard',
              random_seed=86668740)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=H,static_df=df_static,n_windows=None, test_size=test_length-test_length%H,step_size=H)
forecasts = forecasts.dropna()
if "PatchTST-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["PatchTST"].values
else:
  Y_hat=forecasts["PatchTST-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

if "PatchTST-median" not in list(forecasts.columns.values):
  df2=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")
  df2=df2.pivot(columns="unique_id")
else:
  df2=forecasts[["ds","unique_id","PatchTST-median"]].set_index("ds")
  df2=df2.pivot(columns="unique_id")

df2.to_excel("PatchTST forecasts.xlsx")

INFO:lightning_fabric.utilities.seed:Seed set to 86668740


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.005477468040900361 | QLIKE: 0.05594005733238158 | MAE: 0.0033761315431171408 | MAPE: 0.25713652533486403
